In [2]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
def cagematch_feud_scraper(number):
    #headers parameter makes sure the site is readable
    res = requests.get(f'https://www.cagematch.net/?id=87&s={number}', headers={'Accept-Encoding': 'identity'})
   #prints status_code everytime the function runs correctly
    print(f'res:{res.status_code}')
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find_all("tr")
    #empty list to hold every row
    feuds = []
    #iterates over whole page, retrieving the data I need for my dataset
    for row in table[1:]:
        feud = {}
        feud['promotion'] = row.find_all('td')[1].find('img').attrs['title']
        feud['title'] = row.find_all('td')[2].find('a').text
        feud['start_date'] = row.find_all('td')[3].text.replace(u'\xa0', u' ').split('-')[0]
        feud['end_date'] = row.find_all('td')[3].text.replace(u'\xa0', u' ').split('-')[1]
        #try statment needed here if the rating column is empty, which only happens later in the
        #site's sorting of data.  
        try:
            feud['rating'] = float(row.find_all('td')[4].text)
        except:
            print('something went wrong with the ratings')
            break
        #going into the hyperlinked page for the feud itself, to get participants
        sub_res = requests.get(url = 'https://www.cagematch.net/?id=87/' 
                            + row.find_all('td')[2].find('a').attrs['href'],
                           headers={'Accept-Encoding': 'identity'})
        print(f'subres: {sub_res.status_code}')
        sub_soup= BeautifulSoup(sub_res.content, 'lxml')
        sub_table = sub_soup.find_all('a')
        #not every feud has 4 participants, but this range will catch all members.  later cleaning
        #will remove noise picked up here.
        feud['party_1'] = sub_table[81].text
        feud['party_2'] = sub_table[82].text
        feud['party_3'] = sub_table[83].text
        feud['party_4'] = sub_table[84].text
        feuds.append(feud)
    return feuds

In [4]:
datalist = []
#scrapes every page where matches have proper ratings.  this will run for a WHILE.
for num in [0,100,200,300,400,500]:
    datalist.append(pd.DataFrame(cagematch_feud_scraper(num)))
df = pd.concat(datalist, axis = 0)
df.head()
#saves our data to a csv, to be cleaned and EDA'ed in a different notebook
df.to_csv('./cagematch_dataframe.csv', index = False)

res:200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 200
subres: 

,promotion,title,start_date,end_date,rating,parties
0,World Wrestling Entertainment,Legends of WrestleMania Teil II: Die Obsession...,12/2009,03/2010,9.87,CommentsShawn MichaelsThe UndertakerWorld Wres...
1,Ring Of Honor,I hate your f***ing guts: Generico vs. Steen,12/2009,12/2010,9.87,El GenericoColt CabanaKevin SteenSteve Corino
2,World Wrestling Entertainment,Die Abenteuer der SmackDown Six,10/2002,12/2002,9.85,Chris BenoitKurt AngleEddie GuerreroChavo Guer...
3,Ring Of Honor,Heroes of Honor: Samoa Joe vs. CM Punk,06/2004,12/2004,9.85,Samoa JoeCM PunkRing Of HonorSummary
4,World Wrestling Entertainment,"Drei Teams, drei Buchstaben, ein Mythos: Hardy...",12/1999,05/2001,9.78,CommentsJeff HardyMatt HardyLita
